# Vježba 3 - Vektorska reprezentacija dokumenta

<ol><li> Kreirati promjenljivu <b>str1</b> koja sadrži tekst 'zdravo', te listu <b>str_list</b> koja sadrži stringove: 'zdravo', 'zbogom', 'zdravo', 'zbogom', 'zdravo', 'zdravoratumski'. Napisati kod koji broji koliko puta se string 'zdravo' nalazi u listi <b>str_list</b>.
</li></ol>

In [1]:
#KOD ZA ZADATAK 1
str1 = 'zdravo'
str_list = ['zdravo', 'zbogom', 'zdravo', 'zbogom', 'zdravo', 'zdravoratumski']
counter = 0
for str_entity in str_list:
    if str1 in str_entity:
        counter += 1
print('Rijec', str1, 'se pojavljuje', counter, 'puta u listi str_list')

Rijec zdravo se pojavljuje 4 puta u listi str_list


<ol start = "2">
<li> Napisati kod koji se u listi <b>str_list</b> pronalazi string najsličniji riječi 'zdravorazumski', pri čemu se sličnost mjeri brojem istih slova.
</li>
</ol>

In [2]:
#KOD ZA ZADATAK 2
import numpy as np
import sys

word_to_find = 'zdravorazumski'

def getOccurance(text):
    lower_case = np.zeros(26)
    for character in text:
        lower_case[ord(character) - ord('a')] += 1
    return lower_case

zdrm = getOccurance(word_to_find)

index_of_most_similar = -1
similarity_value = sys.maxsize
print('Najveci moguci broj je', sys.maxsize)

for i, string in enumerate(str_list):
    occurances = getOccurance(string)
    temp_similarity = np.abs(occurances - zdrm).sum()
    if temp_similarity < similarity_value:
        similarity_value = temp_similarity
        index_of_most_similar = i
print('Najslicniji', word_to_find, 'je', str_list[index_of_most_similar])


slicna_rijec = 'zdravorazumski'
counters = [0, 0, 0, 0, 0, 0]
counter = 0
for rijec in str_list:
    for i in range(len(rijec)):
        if rijec[i] in slicna_rijec:
            counters[counter] += 1
    print('Rijec', rijec, 'ima', counters[counter], 'istih slova kao i rijec', slicna_rijec)
    counter += 1

Najveci moguci broj je 9223372036854775807
Najslicniji zdravorazumski je zdravoratumski
Rijec zdravo ima 6 istih slova kao i rijec zdravorazumski
Rijec zbogom ima 4 istih slova kao i rijec zdravorazumski
Rijec zdravo ima 6 istih slova kao i rijec zdravorazumski
Rijec zbogom ima 4 istih slova kao i rijec zdravorazumski
Rijec zdravo ima 6 istih slova kao i rijec zdravorazumski
Rijec zdravoratumski ima 13 istih slova kao i rijec zdravorazumski


<ol start = "3">
<li> Kreirajte promjenljivu koja sadrži string 'eci peci pec ti si mali zec'. Tokenizujte ovu promjenljivu na riječi koje sadrži. Rezultat treba da bude lista koja sadrži pojedine riječi zadatog stringa.
</li>
</ol>

In [3]:
#KOD ZA ZADATAK 3
sentence = 'eci peci pec ti si mali zec'
token = sentence.split()
print(type(token), '\n', token)

rabbit_tokens = { x for x in sentence.split() }
print(type(rabbit_tokens), '\n', rabbit_tokens)

<class 'list'> 
 ['eci', 'peci', 'pec', 'ti', 'si', 'mali', 'zec']
<class 'set'> 
 {'si', 'pec', 'zec', 'eci', 'mali', 'peci', 'ti'}


U narednom dijelu vježbe ćemo iskoristiti korpus dokumenata sa projekta Gutenberg da bismo kreirali njihovu vektorsku reprezentaciju, a nakon toga implementirali sistem koji za proizvoljan dokument pronalazi najsličniji dokument iz Gutenberg korpusa. U narednoj ćeliji koda su date dvije funkcije koje se mogu iskoristiti za tokenizaciju dokumenta.

In [4]:
from nltk.stem.porter import PorterStemmer
import re
import nltk

stemmer = PorterStemmer()
minlen = 1

def normalize(word):
    # Funkcija koja vrši normalizaciju riječi
    word = re.sub('[^A-Za-z0-9]+', '', word)    #Ukljanjanje svih karakter koji nisu alfanumerički
    word = re.sub('[0-9]+', 'number', word)     #Zamjena svih brojeva u tekstu riječju number
    word = word.lower()                         #Pretvaranje u mala slova
    return word

def tokenize(text):
    #Funkcija koja vrši tokenizaciju teksta na sastavne riječi
    tokens = nltk.word_tokenize(text)           #Tokenizacija teksta
    stems = []
    for token in tokens:
        word = normalize(token)                 #Normalizacija pronađenih tokena
        stem = stemmer.stem(word)               #Stemizacija tokena
        if len(stem) > minlen:                  #Zadržavanje tokena čija je dužina veća od minlen
            stems.append(stem)
    return stems

Potrebno je napisati kod koji će za sve dokumente iz Gutenberg korpusa izračunati njihovu vektorsku reprezentaciju i to sačuvati u promjenljivu representations. Da bi se to uradilo potrebno je:
1. Za svaki dokument odrediti tokene,
2. Odrediti rječnik za dati korpus,
3. Za svaki dokument na osnovu rječnika odrediti vektorsku reprezentaciju.
U narednoj ćeliji su date kostur potrebnog koda.

In [5]:
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

#Putanja do korpusa
path = './gutenberg'

#Dictionary koji će za svaki dokument sadržati njegove tokene
token_dict = {}

#Prolazak kroz sve fajlove
for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)    #Putanja do fajla
        #Izršiti učitavanje fajla i njegovu tokenizaciju
        #Tokene svakog fajla upisati u token_dict i to tako da je ključ naziv fajla, a vrijednost tokeni
        #KOD
        with open(fname, 'r') as file:
            text = file.read()
            tokens = tokenize(text)
            token_dict[fname] = tokens

#Lista koja će predstavljati rječnik svih rječi iz korpusa
dictionary = []
stop_words = set(stopwords.words('english'))

#Proći kroz tokene za sve fajlove i dodati ih u dictionary, obratiti pažnju da često korištene riječi (stop_words) nije potrebno
#dodavati u rječnik

#KOD

for tokens in token_dict.values():
    for token in tokens:
        if not token in dictionary and not token in stop_words:
            dictionary.append(token)
            
#Kako rječnik ne bi trebalo da sadrži ponovljene riječi potrebno je ostaviti samo jedinstvene riječi

#KOD

# odradjeno u if uslovu gore

#Kako bi se ubrzalo pretraživanje rječnika, rječnik se pretvara u dictionary koji za ključeve ima riječi, a za vrijednosti
#indeks koji odgovara poziciji te riječi u reprezentaciji

dictionary_dict = {word: i for i, word in enumerate(dictionary)}

print(len(dictionary_dict.keys()))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
29670


In [ ]:
import numpy as np

def compute_document_representation(tokens):
    #Napisati funkciju koja kreira reprezentaciju dokumenta na osnovu ulaznih tokena i to tako da se za svaku riječ iz rječnika
    #odredi koliko puta se pojavljuje u dokumentu
    #Izlaz treba biti niz ranga 1 dužine koja odgovara dužini rječnika
    #KOD
    #pass
    vector = np.zeors(len(dictionary_dict))
    for token in tokens:
        if token in dictionary_dict:
            vector[dictionary_dict[token]] += 1
    return vector
    
#Kreiranje numpy niza koji će sadržati reprezentacije svakog dokumenta iz korpusa, shape je (broj_dokumenata, broj_rječi)
representations = np.zeros((len(token_dict), len(dictionary_dict)))

#Kreirati reprezentaciju svih dokumenta iz korpusa tako što ćete iskoristiti prethodno napisanu funkciju

#KOD
for i, tokens in enumerate(token_dict.values()):
    representations[i] = compute_document_representation(tokens)

Nakon što smo odredili reprezentaciju za sve dokumente ove reprezentacije je moguće iskoristiti za pretraživanje. Pretraživanje izvršiti na način da se za proizvoljno izabran dokument pronalaze tri najsličnija dokumenta iz korpusa. Za računanje sličnosti iskoristiti i SSD i kosinusnu sličnost. Za računanje kosinusne sličnosti je moguće iskoristiti funkciju cosine_similarity iz modula sklearn.metrics.pairwise.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#KOD
doc_id = 0

sims = cosine_similarity(representations[doc_id].reshape(1, -1), representations)
ssd = { ((a - representations[doc_id]) ** 2).sum(): i for i, a in enumerate(representations) }

Za računanje TFIDF reprezentacije dokumenta moguće je iskoristiti sklearn biblioteku i kod dat u narednoj ćeliji.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import string

#Putanja do korpusa
path = './gutenberg'

#Dictionary koji sadrži tekst svakog dokumenta
text_dict = {}

#Čitanje sadržaja svih dokumenata
for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)
        print (fname)
        with open(fname) as file:
            text_dict[f] = file.read()

#Definisanje objekta koji služi za određivanje TFIDF reprezentacije, za tokenizaciju se koristifunkcije data
#kao parametar tokenize, dok se kao stop riječi koriste one iz enegleskog jezika (moguće je zadati i listu koja sadrži stop riječi)
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')

#Učenje rječnika i transformacija dokumenata koji su iskorišteni za učenje rječnika
#U representations_tfidf se sada nalazi rezrezentacija svih dokumenata iz korpusa
representations_tfidf = tfidf.fit_transform(text_dict.values())

str = 'all great and precious things are lonely.'

#Određivanje reprezentacije za novi dokument
#Treba obratiti pažnju da su reprezentacije date kao sparse matrice radi efikasnijg čuvanja u memoriji ali i bržeg procesiranja.
response = tfidf.transform([str])

response

Za proizvoljan dokument odrediti 3 najsličnija dokumenta iz korpusa na osnovu SSD i kosinusne sličnosti. Prethodno navedenu funkciju za kosinusnu sličnost je moguće koristiti i sa sparse matricama.

In [ ]:
#KOD

sort_args_sims = sims.argsort()[0][::-1]
sort_args_ssd = sorted(ssd)

for i in sort_args_sims[:3:1]:
    print(list(token_dict)[1])
    
print()

for i in sort_args_ssd